# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
city_data = pd.read_csv('city_data-Copy1.csv')
city_data.dropna()
city_data.head()


,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,narsaq,60.9167,-46.0500,32.22,36,2,7.58,GL,1634563253
1,mataura,-46.1927,168.8643,52.70,60,99,9.35,NZ,1634563254
2,georgetown,5.4112,100.3354,84.15,76,20,0.00,MY,1634563254
3,listvyanskiy,54.4399,83.4984,45.19,76,5,6.98,RU,1634563255
4,mar del plata,-38.0023,-57.5575,57.99,27,0,8.01,AR,1634563108


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# configure gmaps
gmaps.configure(api_key=gkey)

In [6]:
# using lat and lon from data frame to identify locations of cities
cities = city_data[['Lat', 'Lon']]

# identifying the humidity from the data frame above to add weight to the map
humidity = city_data['Humidity'].astype(float)


In [9]:
# adding heat layer to the map
heat_map = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(cities, weights=humidity, 
                                    dissipating=False, max_intensity=100,
                                   point_radius=2)

# add the heat map layer to the heat map figure
heat_map.add_layer(heatmap_layer)

#display the heatmap
heat_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
# narrow down cities for ideal weather conditions which include:

# max temp lower than 80, greater than 70
ideal_temp = city_data[(city_data['Max Temp'] > 70) & (city_data['Max Temp'] < 80)]

# wind speed less than 10mph
wind_speed = ideal_temp[ideal_temp['Wind Speed'] < 10]

# zero cloudiness
zero_clouds = wind_speed[wind_speed['Cloudiness'] == 0]

# drop any rows that do not contain all three conditions
zero_clouds = zero_clouds.dropna()
zero_clouds

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,cape town,-33.9258,18.4232,70.11,42,0,8.01,ZA,1634563104
170,san patricio,28.0170,-97.5169,71.83,74,0,6.91,US,1634563308
191,agropoli,40.3469,14.9966,71.87,54,0,5.28,IT,1634563314
284,rudbar,30.1500,62.6000,74.43,7,0,4.68,AF,1634563345
312,vallenar,-28.5708,-70.7581,71.40,33,0,3.24,CL,1634563354
327,sao filipe,14.8961,-24.4956,79.93,73,0,8.48,CV,1634563360
355,los llanos de aridane,28.6585,-17.9182,78.55,30,0,4.61,ES,1634563369
385,moranbah,-22.0016,148.0466,77.04,20,0,9.62,AU,1634563379
420,chikhli,20.3500,76.2500,75.04,78,0,7.52,IN,1634563392
421,calvinia,-31.4707,19.7760,71.02,12,0,3.04,ZA,1634563392


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
# store ideal cities in variable named hotel_df
hotel_df = zero_clouds

# add a "Hotel Name" column to the data frame
hotel_df['Hotel Name'] = ""

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set parameters for hotel to search within 5000 meters
params = {'radius' : 5000,
          'types' : 'hotel',
          'keyword' : 'hotels',
          'key' : gkey}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    # get city location from dataframe use lat and lon
    lat = row['Lat']
    lon = row['Lon']
    
    # add location to params dictionary using lat and lon
    params['location'] = f'{lat},{lon}'
    
    # assemble url and make API request
    print(f'Retrieving Results for Index {index}.')
    response = requests.get(base_url, params=params).json()
    
    # extract the results
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        
    except:
        print('Issues with finding a hotel at', row['City'])
        hotel_df.loc[index, 'Hotel Name'] = 'NA'
        
hotel_df

Retrieving Results for Index 37.
Retrieving Results for Index 170.
Retrieving Results for Index 191.
Retrieving Results for Index 284.
Issues with finding a hotel at rudbar
Retrieving Results for Index 312.
Retrieving Results for Index 327.
Retrieving Results for Index 355.
Retrieving Results for Index 385.
Retrieving Results for Index 420.
Retrieving Results for Index 421.
Retrieving Results for Index 472.
Retrieving Results for Index 539.


,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
37,cape town,-33.9258,18.4232,70.11,42,0,8.01,ZA,1634563104,POD Camps Bay
170,san patricio,28.0170,-97.5169,71.83,74,0,6.91,US,1634563308,Americas Best Value Inn Sinton
191,agropoli,40.3469,14.9966,71.87,54,0,5.28,IT,1634563314,Hotel Residence Serenella
284,rudbar,30.1500,62.6000,74.43,7,0,4.68,AF,1634563345,NA
312,vallenar,-28.5708,-70.7581,71.40,33,0,3.24,CL,1634563354,Hotel Solaris
327,sao filipe,14.8961,-24.4956,79.93,73,0,8.48,CV,1634563360,Casas Do Sol
355,los llanos de aridane,28.6585,-17.9182,78.55,30,0,4.61,ES,1634563369,Hotel Hacienda de Abajo
385,moranbah,-22.0016,148.0466,77.04,20,0,9.62,AU,1634563379,Direct Hotels - Monterey Moranbah
420,chikhli,20.3500,76.2500,75.04,78,0,7.52,IN,1634563392,Meera Celebrations
421,calvinia,-31.4707,19.7760,71.02,12,0,3.04,ZA,1634563392,Calvinia Hotel by Country Hotels


In [17]:
# delete any rows where hotel was not found
hotel_df = hotel_df[hotel_df['Hotel Name'] != 'NA']
hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
37,cape town,-33.9258,18.4232,70.11,42,0,8.01,ZA,1634563104,POD Camps Bay
170,san patricio,28.0170,-97.5169,71.83,74,0,6.91,US,1634563308,Americas Best Value Inn Sinton
191,agropoli,40.3469,14.9966,71.87,54,0,5.28,IT,1634563314,Hotel Residence Serenella
312,vallenar,-28.5708,-70.7581,71.40,33,0,3.24,CL,1634563354,Hotel Solaris
327,sao filipe,14.8961,-24.4956,79.93,73,0,8.48,CV,1634563360,Casas Do Sol
355,los llanos de aridane,28.6585,-17.9182,78.55,30,0,4.61,ES,1634563369,Hotel Hacienda de Abajo
385,moranbah,-22.0016,148.0466,77.04,20,0,9.62,AU,1634563379,Direct Hotels - Monterey Moranbah
420,chikhli,20.3500,76.2500,75.04,78,0,7.52,IN,1634563392,Meera Celebrations
421,calvinia,-31.4707,19.7760,71.02,12,0,3.04,ZA,1634563392,Calvinia Hotel by Country Hotels
472,kuruman,-27.4524,23.4325,79.00,4,0,9.53,ZA,1634563407,Kuruman Hotel by Country Hotels


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [27]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, hotel_info)

heat_map.add_layer(marker_layer)

# Display figure
heat_map

Figure(layout=FigureLayout(height='420px'))